In [198]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torch import nn
import cv2
import pytorch_lightning as pl
import torch
from sklearn.model_selection import train_test_split

In [199]:
input_df = "../Data/Quadratic Equation Full Details.csv"
dir_path = "../Data/Images of Equations"
df = pd.read_csv(input_df)
df

,id,path,a,b,c,root_1,root_2,equation
0,eqn_0000,quadratic/eqn_0000,1,7,4,-0.6277,-6.3723,x² + 7x + 4 = 0
1,eqn_0001,quadratic/eqn_0001,15,-22,-76,3.1007,-1.6340,15x² - 22x = 76
2,eqn_0002,quadratic/eqn_0002,12,33,22,-1.1356,-1.6144,33x + 22 = -12x²
3,eqn_0003,quadratic/eqn_0003,14,-10,-25,1.7404,-1.0261,14x² - 10x = 25
4,eqn_0004,quadratic/eqn_0004,4,10,-6,0.5000,-3.0000,4x² + 10x - 6 = 0
...,...,...,...,...,...,...,...,...
1995,eqn_1995,quadratic/eqn_1995,5,38,9,-0.2447,-7.3553,38x + 9 = -5x²
1996,eqn_1996,quadratic/eqn_1996,17,-24,4,1.2187,0.1931,17x² - 24x = -4
1997,eqn_1997,quadratic/eqn_1997,2,8,-5,0.5495,-4.5495,2x² + 8x - 5 = 0
1998,eqn_1998,quadratic/eqn_1998,1,9,7,-0.8599,-8.1401,x² + 9x + 7 = 0


In [200]:
# transformers = transforms.Compose([
#     transforms.
# ])

In [201]:
class Quadratic_Dataset(Dataset):
    def __init__(self,path,targets = None):
        self.path =  path
        self.targets = targets
        self.dir_path = "../Data/Images of Equations/"
    
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, index):
        image_path = self.dir_path + self.path[index] + ".png"
        image = cv2.imread(image_path)
        image = cv2.resize(image,(100,100))
        if self.targets is None:
            return torch.tensor(image).float().reshape(3,100,100)/256
        else:
            return torch.tensor(image).float().reshape(3,100,100)/256 , torch.tensor(self.targets[index]).float()


Sanity Check to Make Sure of what you are sending to the model

In [202]:
dataset = Quadratic_Dataset(path = df["id"].values , targets = df["a"].values)
next(iter(DataLoader(dataset , batch_size = 1 , shuffle = True)))

[tensor([[[[0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
           [0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
           [0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
           ...,
           [0.1797, 0.1797, 0.0117,  ..., 0.9961, 0.9961, 0.9961],
           [0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
           [0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961]],
 
          [[0.9961, 0.9961, 0.0000,  ..., 0.9961, 0.9961, 0.9961],
           [0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
           [0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
           ...,
           [0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
           [0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
           [0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961]],
 
          [[0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
           [0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
           [0.9961, 0.99

In [203]:
train_df , test_df = train_test_split(df,test_size = 0.33 , random_state= 42)
train_df , val_df = train_test_split(test_df,test_size = 0.33 , random_state= 42)

In [204]:
class Quadratic_Module(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.train_dataset = Quadratic_Dataset(path = train_df["id"].values , targets = train_df["a"].values)
        self.test_dataset = Quadratic_Dataset(path = test_df["id"].values , targets = test_df["a"].values)
        self.val_dataset = Quadratic_Dataset(path = val_df["id"].values , targets = val_df["a"].values)

    def prepare_data(self) :
        pass
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset , batch_size = 32  , shuffle = True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset , batch_size = 32  , shuffle = False)

    def val_dataloader(self):
        return DataLoader(self.val_dataset , batch_size = 32  , shuffle = False)


sanity check

In [205]:
x = torch.ones(1,3,100*100)
linear = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3*100*100 , 1)

        )
linear(x)

tensor([[0.8942]], grad_fn=<AddmmBackward>)

In [211]:
class Quadratic_Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3*100*100 ,10),
            nn.LeakyReLU(0.33),
            nn.Linear(10,1)

        )
        self.loss_func = nn.MSELoss()
        
    def forward(self,x):
        out = self.linear(x)
        return out
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = 1e-4)
        return optimizer

    def training_step(self,batch,batch_idx):
        x , y_true = batch
        y_pred = self(x)
        loss = self.loss_func(y_true, y_pred)
        self.log("train_loss" , loss)
        return loss
    
    def test_step(self,batch,batch_idx):
        x , y_true = batch
        y_pred = self(x)
        loss = self.loss_func(y_true, y_pred)
        self.log("test_loss" , loss)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x , y_true = batch
        y_pred = self(x)
        loss = self.loss_func(y_true, y_pred)
        self.log("val_loss" , loss)
        return loss



In [212]:
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(monitor= "val_loss" , 
mode = "min"
)

In [213]:
from pytorch_lightning import Trainer
model = Quadratic_Model()
module = Quadratic_Module()
trainer = Trainer(max_epochs = 10,callbacks=[checkpoint])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [214]:
trainer.fit(model,module)


  | Name      | Type       | Params
-----------------------------------------
0 | linear    | Sequential | 300 K 
1 | loss_func | MSELoss    | 0     
-----------------------------------------
300 K     Trainable params
0         Non-trainable params
300 K     Total params
1.200     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\Keegan Fernandes\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/21 [00:00<00:00, 500.87it/s]

C:\Users\Keegan Fernandes\anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Keegan Fernandes\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\Keegan Fernandes\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:322: UserWarning: The number of training samples (14) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps i

Epoch 0:  67%|██████▋   | 14/21 [00:02<00:01,  5.16it/s, loss=42.7, v_num=10]

C:\Users\Keegan Fernandes\anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([26, 1])) that is different to the input size (torch.Size([26])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 9: 100%|██████████| 21/21 [00:02<00:00,  8.30it/s, loss=37.9, v_num=10]


In [ ]:
trainer.test()
